# Faiss IndexFlatL2演示

[官网文档](https://github.com/facebookresearch/faiss/wiki)
[Faiss-indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes)

## 目录
- 构造测试数据
- 创建索引并添加训练样本向量集合
- 用k-近邻搜索

In [29]:
import numpy as np
import faiss
import time

下面构造如何向量：
1. 1万个数据
2. 每个数据64个维度

## 构造测试数据

Faiss处理固定维数d的向量集合，d通常为几十到几百，存储在矩阵中。

In [68]:
d = 64                           
nb = 100000                       
nq = 10000                       
np.random.seed(1234)                              # 对生成的数据，每次都一样，可预见，若无此句，每次都不一样
xb = np.random.random((nb, d)).astype('float32')  # 向量集合，矩阵nb*d
xb[:, 0] += np.arange(nb) / 1000                  # 修正向量集合第一列，每项加 0...nb/1000 小数
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000 

> 说明：
>
> `xb`为`[100000,64]`的训练数据，`xq`为`[10000,64]`的查询数据。

行主存储，即向量编号为`i`的第`j`个分量存储在矩阵的第`i`行第`j`列中。

例如，如下向量集合为5个数据，每个数据为2维度：
```
[[0.19151945 0.62210876]
 [0.43872774 0.7853586 ]
 [0.7819758  0.2725926 ]
 [0.27946424 0.8018722 ]
 [0.96213937 0.87593263]]
```
Faiss只使用32位浮点矩阵。

## 创建索引并添加训练样本向量集合

faiss创建索引从而对向量预处理，提高查询效率。

faiss提供多种索引方法，最简单的暴力检索L2距离：`IndexFlatL2`。

当创建索引时，必须指定数据集向量的维度`d`。 

大多数索引还需要训练来分析向量集的分布。但`IndexFlatL2`可以跳过这个操作。

In [13]:
index = faiss.IndexFlatL2(d)
print('is_trained',index.is_trained)
index.add(xb)
print('ntotal', index.ntotal)

> 说明：
> 
> 将向量 xb 添加到 index。

## 用k-近邻搜索

In [34]:
k = 4   # we want to see 4 nearest neighbors
# D, I = index.search(xb[:5], k) # sanity check
# # print(xb[:5])
# print(I)     # 向量索引位置
# print(D)     # 相似度矩阵

D, I = index.search(xq, 10)     # actual search
# xq is the query data
# k is the num of neigbors you want to search
# D is the distance matrix between xq and k neigbors
# I is the index matrix of k neigbors
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:]) # neighbors of the 5 last queries

#从index中恢复数据，indexFlatL2索引就是将向量进行排序
# print(xb[381])
# print(index.reconstruct(381))

[[50794   794 10794 30794 40794 20794 30072 40072 20072    72]
 [40393 30393 50393 10393 20393   393 10919   919 40919 30919]
 [40688 50688 10688   688 30688 20688 10806   806 40806 30806]
 [50950 10950 20950   950 40950 30950 30230 40230 20230 10230]
 [30199 50199   199 20199 40199 10199 10409   409 40409 30409]]
[[28664  8664 38664 58664 48664 18664  9047 49047 39047 29047]
 [59914 19914  9914 39914 49914 29914  9767 49767 39767 29767]
 [38853 58853 18853 28853 48853  8853 19539  9539 49539 39539]
 [39352 59352  9352 29352 49352 19352 19411  9411 49411 39411]
 [59936 19936  9936 39936 49936 29936  9592 49592 39592 19592]]


add方法一般添加训练时的样本，search就是寻找相似向量。

一些索引可以保存整型的ID，每个向量可以指定一个ID，当查询相似向量时，会返回相似向量的ID及相似度(或距离)。

如果不指定，将按照添加的顺序从0开始累加。其中IndexFlatL2不支持指定ID。

```python
True
1000000
```